In [550]:
import numpy as np
from tqdm import tqdm
import random
import re

In [586]:
with open('aoc19_input.txt') as f:
    lines = f.readlines()
lines = [list(map(int,re.findall(r'\d+', line))) for line in lines]
puzz_input = lines
puzz_input

[[1, 4, 3, 2, 14, 4, 11],
 [2, 3, 3, 3, 20, 2, 12],
 [3, 4, 4, 3, 20, 2, 10],
 [4, 4, 3, 4, 16, 2, 15],
 [5, 4, 4, 3, 7, 4, 20],
 [6, 2, 4, 4, 19, 2, 18],
 [7, 2, 3, 2, 14, 3, 20],
 [8, 2, 4, 4, 16, 4, 17],
 [9, 4, 3, 3, 11, 4, 7],
 [10, 4, 3, 2, 17, 3, 16],
 [11, 2, 4, 3, 19, 4, 12],
 [12, 3, 3, 2, 16, 2, 18],
 [13, 2, 3, 3, 13, 2, 20],
 [14, 3, 4, 3, 16, 3, 14],
 [15, 4, 4, 4, 17, 4, 16],
 [16, 4, 4, 4, 8, 3, 19],
 [17, 3, 3, 3, 19, 3, 19],
 [18, 3, 3, 3, 11, 2, 8],
 [19, 4, 4, 2, 7, 3, 10],
 [20, 3, 4, 4, 18, 4, 12],
 [21, 3, 3, 3, 19, 3, 17],
 [22, 4, 4, 4, 20, 2, 12],
 [23, 4, 3, 3, 7, 3, 9],
 [24, 2, 2, 2, 17, 2, 10],
 [25, 4, 3, 2, 15, 2, 8],
 [26, 4, 4, 2, 14, 3, 17],
 [27, 3, 4, 3, 19, 3, 8],
 [28, 3, 3, 3, 8, 2, 12],
 [29, 4, 3, 2, 19, 3, 13],
 [30, 3, 4, 3, 6, 4, 11]]

In [593]:
class RobotFactory:
    def __init__(self,blueprint):
        self.bpid = blueprint[0]
        self.num_ores = 0
        self.num_orebots = 1
        self.orebot_ore_cost = blueprint[1]

        self.num_clay = 0
        self.num_claybots = 0
        self.claybot_ore_cost = blueprint[2]

        self.num_obs = 0
        self.num_obbots = 0
        self.obbot_ore_cost = blueprint[3]
        self.obbot_clay_cost = blueprint[4]

        self.num_geodes = 0
        self.num_geodebots = 0
        self.geodebot_ore_cost = blueprint[5]
        self.geodebot_obs_cost = blueprint[6]
        
        self.clock = 0
        
        
    def rate_calculations(self):
        orebot_ore_turns = (self.orebot_ore_cost - self.num_ores) / self.num_orebots if self.num_orebots > 0 else np.inf
        claybot_ore_turns = (self.claybot_ore_cost - self.num_ores) / self.num_orebots if self.num_orebots > 0 else np.inf
        obbot_clay_turns = (self.obbot_clay_cost - self.num_clay) / self.num_claybots if self.num_claybots > 0 else np.inf
        geodebot_obs_turns = (self.geodebot_obs_cost - self.num_obs) / self.num_obbots if self.num_obbots > 0 else np.inf
        geodebot_ore_turns = (self.geodebot_ore_cost - self.num_ores) / self.num_orebots
        obbot_ore_turns = (self.obbot_ore_cost - self.num_ores) / self.num_orebots
        return geodebot_obs_turns, geodebot_ore_turns, obbot_clay_turns, obbot_ore_turns, claybot_ore_turns, orebot_ore_turns
    
    def next_robot(self):
        geodebot_obs_turns, geodebot_ore_turns, obbot_clay_turns, obbot_ore_turns, claybot_ore_turns, orebot_ore_turns = self.rate_calculations()
        if self.num_ores >= self.geodebot_ore_cost and self.num_obs >= self.geodebot_obs_cost:
            return 'geodebot'
        elif self.num_ores >= self.obbot_ore_cost and self.num_clay >= self.obbot_clay_cost:
            return 'obbot'
        elif self.num_ores >= self.claybot_ore_cost:
            if obbot_clay_turns > obbot_ore_turns or self.num_claybots == 0:
                return 'claybot'
        elif self.num_ores >= self.orebot_ore_cost:
            if orebot_ore_turns == max([geodebot_obs_turns, obbot_clay_turns, claybot_ore_turns, orebot_ore_turns]):
                return 'orebot'
    def next_robot_mc(self):
        if self.num_ores >= self.geodebot_ore_cost and self.num_obs >= self.geodebot_obs_cost:
            return 'geodebot'
        if self.num_ores >= self.obbot_ore_cost and self.num_clay >= self.obbot_clay_cost:
            if random.choice([True, False]):
                return 'obbot'
        if self.num_ores >= self.claybot_ore_cost:
            if random.choice([True, False]):
                return 'claybot'
        if self.num_ores >= self.orebot_ore_cost:
            if random.choice([True, False]):
                return 'orebot'
    def spend_resources(self,next_bot):
        if next_bot == 'geodebot':
            self.num_ores -= self.geodebot_ore_cost
            self.num_obs -= self.geodebot_obs_cost
        elif next_bot == 'obbot':
            self.num_ores -= self.obbot_ore_cost
            self.num_clay -= self.obbot_clay_cost
        elif next_bot == 'claybot':
            self.num_ores -= self.claybot_ore_cost
        elif next_bot == 'orebot':
            self.num_ores -= self.orebot_ore_cost        
        
    def collect_resources(self):
        self.num_ores += self.num_orebots
        self.num_clay += self.num_claybots
        self.num_obs += self.num_obbots
        self.num_geodes += self.num_geodebots
        
    def finish_robot(self,next_bot):
        if next_bot == 'geodebot':
            self.num_geodebots += 1
        elif next_bot == 'obbot':
            self.num_obbots += 1
        elif next_bot == 'claybot':
            self.num_claybots += 1
        elif next_bot == 'orebot':
            self.num_orebots += 1
            
    def iterate(self):
#         next_bot = self.next_robot()
        next_bot = self.next_robot_mc()
        self.spend_resources(next_bot)
        self.collect_resources()
        self.finish_robot(next_bot)
        self.clock += 1
    def printout_stats(self):
        print('time:',self.clock)
        print("Ores:",self.num_ores)
        print('Orebots:',self.num_orebots)
        print("Clays:",self.num_clay)
        print('Claybots:',self.num_claybots)
        print("Obs:",self.num_obs)
        print('Obbots:',self.num_obbots)
        print("Geodes:",self.num_geodes)
        print('Geodebots:',self.num_geodebots)
        geodebot_obs_turns, geodebot_ore_turns, obbot_clay_turns, obbot_ore_turns, claybot_ore_turns, orebot_ore_turns = self.rate_calculations()
#         print("Geodebot turn to obsidian:", geodebot_obs_turns)
#         print("Obsidianbot turns to clay:", obbot_clay_turns)
#         print("Naive Geodebot turns to Ore:",  geodebot_ore_turns)
#         print("Naive Obsidianbot turns to Ore:",  obbot_ore_turns)
#         print("Naive Claybot turns to Ore:",  claybot_ore_turns)
#         print("Naive Orebot turns to Ore:",  orebot_ore_turns)
        
        
        
        
        

In [594]:
example_bps = [[1,4,2,3,14,2,7],[2,2,3,3,8,3,12]]

In [619]:
def do_mc_factory(n,steps,bp):
    geodes = []
    for i in tqdm(range(n)):
        factory = RobotFactory(bp)
        for i in range(steps):
            factory.iterate()
        #     factory.printout_stats()
        geodes.append(factory.num_geodes)
    return max(geodes), bp[0]* max(geodes)

In [602]:
n = 100000
qualities = []
for bp in tqdm(puzz_input):
    geodes, quality = do_mc_factory(n,bp)
    qualities.append(quality)
print(sum(qualities))

100%|███████████████████████████████████████████████████████████████████████████████████████████| 30/30 [02:47<00:00,  5.59s/it]

1346


## Part 2

In [604]:
trunc_bps = puzz_input[0:3]
trunc_bps

[[1, 4, 3, 2, 14, 4, 11], [2, 3, 3, 3, 20, 2, 12], [3, 4, 4, 3, 20, 2, 10]]

In [620]:
n = 5000000
steps = 32
max_geodes = []
for bp in tqdm(trunc_bps):
    geodes, quality = do_mc_factory(n,steps,bp)
    max_geodes.append(geodes)
print(np.prod(max_geodes))

  2%|█▏                                                                              | 76748/5000000 [00:06<06:22, 12867.18it/s]


  3%|██▌                                                                            | 159485/5000000 [00:12<06:36, 12203.14it/s]


  5%|███▋                                                                           | 236939/5000000 [00:18<06:36, 12012.03it/s]


  6%|████▉                                                                          | 313156/5000000 [00:25<07:31, 10385.38it/s]


  8%|██████                                                                         | 386699/5000000 [00:31<06:19, 12141.81it/s]


...

 95%|██████████████████████████████████████████████████████████████████████████    | 4750484/5000000 [06:11<00:18, 13317.38it/s]


 97%|███████████████████████████████████████████████████████████████████████████▍  | 4834838/5000000 [06:17<00:12, 13411.15it/s]


 98%|████████████████████████████████████████████████████████████████████████████▋ | 4919712/5000000 [06:23<00:06, 13115.94it/s]


100%|████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [20:02<00:00, 400.99s/it]

7644


In [ ]:
7371